In [ ]:
#安装必要的库
%pip install openai "androguard>=4" Pillow

In [1]:
from androguard.util import set_log

set_log("ERROR")  # 关闭琐碎的DEBUG输出
from androguard.misc import AnalyzeAPK
from androguard.core.apk import APK
import os
import re
import openai

openai.api_key = "sk-xxx"
openai.base_url = "https://xxx/v1/"  # 注意：最后要以斜杠/结尾
model = "gpt-4o"  # 不建议更换其他模型
# model = "claude-3-opus"

openai.default_headers = {"x-foo": "true"}


# https://androguard.readthedocs.io/en/latest/intro/gettingstarted.html

In [2]:
prompt_template = """当前app的名字是：{}，包名是：{}
它有如下的activity：
{}

请你根据这些信息，列举出该app的能够在几步的操作内完成的task，例如：
1. 观看视频
2. 搜索视频
3. 对搜索结果进行筛选、排序
4. 评论视频
5. 投币
6. 点赞视频
7. 收藏视频
8. 点赞评论
9. 回复评论
10. 发送私信

请注意，你只能列举task，不能输出任何多余的信息，并且要尽可能多地列举task。你列举的task应该是用户日常会使用到的功能。
"""

prompt_template2 = """Task Examples:
• 淘宝
- 级别 1: 基础操作 - 搜索一台华为 Mate60。
- 级别 2: 复杂操作 - 添加一台华为 Mate60 至购物车，并确认购物车内有此商品。
- 级别 3: 多项任务组合 - 添加一台 1TB 的华为 Mate60 和它对应的手机壳至购物车，并确认购物车内有这些商品。

• 高德地图
- 级别 1: 基础操作 - 搜索一家附近的麦当劳。
- 级别 2: 复杂操作 - 查找到一家附近麦当劳的步行路线。
- 级别 3: 多项任务组合 - 查找到一家附近麦当劳的步行路线，途径一家附近的肯德基。

请根据上面的样例，构建尽可能多的{}的级别3的操作。

以下是{}的级别1和级别2的操作:
{}

请注意，你只能列举操作，不能输出任何多余的信息，并且要尽可能多地列举操作。你列举的操作应该是用户日常会使用到的功能。
"""  # app name, app name, task_str

In [3]:
def chat(prompt):
    completion = openai.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return completion.choices[0].message.content


def apk_to_simple_task(apk_object):
    def parse_task(text):
        pattern = r"\d+\.+(.*)"
        matches = re.findall(pattern, text)
        for i, match in enumerate(matches):
            matches[i] = match.strip()
        return matches

    a = apk_object
    activity = [act for act in a.get_activities()]
    activity_str = "\n".join(activity)
    prompt = prompt_template.format(a.get_app_name(), a.get_package(), activity_str)
    task = parse_task(chat(prompt))
    return task


def simple_task_to_level3_task(simple_task_list, a):
    def parse_task(text):
        pattern = r"\d+\.+(.*)"
        matches = re.findall(pattern, text)
        for i, match in enumerate(matches):
            matches[i] = match.strip()
        return matches

    task_str = ""
    for i, task in enumerate(simple_task_list):
        task_str += f"{i+1}. {task}\n"
    p = prompt_template2.format(a.get_app_name(), a.get_app_name(), task_str)
    c = chat(p)
    t = parse_task(c)
    return t


def save_to_md(a, tasklist, level3_task_list):
    fn = f"{a.get_package()}.md"
    with open(fn, "w") as f:
        f.write(f"# {a.get_app_name()}\n\n")
        f.write(f"包名：{a.get_package()}\n\n")
        f.write(f"main activity: {a.get_main_activity()}\n\n")
        f.write("## 级别1~2的任务\n\n")
        for i, task in enumerate(tasklist):
            f.write(f"{i+1}. {task}\n\n")
        f.write("## 级别3的任务\n\n")
        for i, task in enumerate(level3_task_list):
            f.write(f"{i+1}. {task}\n\n")


def process_apk(apk_path):
    # a, d, dx = AnalyzeAPK(apk_path)
    a = APK(apk_path)
    tasklist = apk_to_simple_task(a)
    level3_task_list = simple_task_to_level3_task(tasklist, a)
    save_to_md(a, tasklist, level3_task_list)

In [ ]:
folder = rf"D:\nd\hw\level"
fn = "bilibili.apk"

apk_fp = os.path.join(folder, fn)  # apk文件路径

process_apk(apk_fp)

In [1]:
from androguard.util import set_log

set_log("ERROR")  # 关闭琐碎的DEBUG输出
from androguard.core.apk import APK
import os

In [2]:
folder = rf"D:\nd\hw\level"
fn = "QQMusic_13.0.0.11_APKPure.apk"
apk_fp = os.path.join(folder, fn)  # apk文件路径
a = APK(apk_fp)

a.get_app_name(), a.get_package(), a.get_main_activity()

('QQ音乐',
 'com.tencent.qqmusic',
 'com.tencent.qqmusic.activity.AppStarterActivity')

In [1]:
import openai

openai.api_key = "sk-xxx"
openai.base_url = "https://xxx/v1/"  # 注意：最后要以斜杠/结尾
model = "gpt-4o"

openai.default_headers = {"x-foo": "true"}

In [2]:
def chat(prompt):
    completion = openai.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return completion.choices[0].message.content

In [3]:
chat("鲁迅为什么打周树人")

'鲁迅是周树人的笔名，实际上鲁迅和周树人是同一个人。因此鲁迅并不存在“打”周树人的情况。这个问题可能是基于误解或玩笑而提出的。鲁迅是中国现代文学的重要作家，他用笔名发表了许多重要作品。'

In [4]:
chat("如何制作西红柿炒钢丝球")

'It seems you have a query about a dish called "西红柿炒钢丝球" (Tomato Stir-fried with Steel Wool). However, "钢丝球" (Steel Wool) is not an edible item and it might be a misunderstanding or typo. If you meant a different ingredient or dish, could you please clarify? If you are looking for a recipe involving tomatoes and a specific ingredient, let me know and I can help with that.'